In [47]:
import pandas as pd
import numpy as np

In [49]:
# open match file
df = pd.read_csv('../match-data/nwsl-events/2025-04-19_North_Carolina_Courage__vs__Bay.csv')

In [51]:
df.head()

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,timeStamp,lastModified,qualifier,playerId,playerName,keyPass,assist,Squad,Position
0,2803484671,1,34,16,0,0,eka22bj7co8bw5qxa6qlbinf8,1,0.0,0.0,2025-04-19T22:31:25.500Z,2025-04-19T23:30:44Z,"[{'id': 5665108477, 'qualifierId': 197, 'value...",NaN,NaN,NaN,NaN,Bay,Away
1,2803484723,1,34,16,0,0,b4fbmos00gxtk8qnaqsm392p5,1,0.0,0.0,2025-04-19T22:31:31.969Z,2025-04-19T23:24:04Z,"[{'id': 5665003257, 'qualifierId': 30, 'value'...",NaN,NaN,NaN,NaN,North Carolina Courage,Home
2,2803502059,2,32,1,0,0,eka22bj7co8bw5qxa6qlbinf8,1,0.0,0.0,2025-04-19T23:09:44.438Z,2025-04-19T23:09:46Z,"[{'id': 5665103473, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,Bay,Away
3,2803502049,2,32,1,0,0,b4fbmos00gxtk8qnaqsm392p5,1,0.0,0.0,2025-04-19T23:09:44.438Z,2025-04-19T23:09:44Z,"[{'id': 5665103453, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,North Carolina Courage,Home
4,2803502135,4,1,1,0,0,b4fbmos00gxtk8qnaqsm392p5,1,50.0,50.0,2025-04-19T23:09:44.439Z,2025-04-19T23:09:50Z,"[{'id': 5665103845, 'qualifierId': 279, 'value...",54ieoaegysojsey2jmqtr9ucp,B. Pinto,NaN,NaN,North Carolina Courage,Home


In [53]:
# Need passes - typeId 1 (pass)

In [63]:
# for i, row in df.iterrows():
#     qualifier_str = row['qualifier']
#     if pd.notna(qualifier_str):
#         # Extract all qualifierId-value pairs using string operations
#         parts = qualifier_str.split("{'id':")  # Split into individual qualifiers
#         for part in parts[1:]:  # Skip the first empty split
#             if "'qualifierId': 140" in part:
#                 value = part.split("'value': '")[1].split("'")[0]
#                 df.at[i, 'endX'] = float(value)
#             elif "'qualifierId': 141" in part:
#                 value = part.split("'value': '")[1].split("'")[0]
#                 df.at[i, 'endY'] = float(value)

# for i in range(len(df)):
#     df1 = df.iloc[i:i+1,:]
#     j = 0
#     for j in range(len(type_cols)):
#         col = df1[type_cols[j]].values[0]
#         if col == 140:
#             endx = df1.loc[:,'qualifier/%i/value' %j].values[0]
#             df['endX'][i] = endx
#         else:
#             j +=1
#     k = 0
#     for k in range(len(type_cols)):
#         col = df1[type_cols[k]].values[0]
#         if col == 141:
#             endy = df1.loc[:,'qualifier/%i/value' %k].values[0]
#             df['endY'][i] = endy
#         else:
#             k +=1

# unpack the qualifiers
type_cols = [col for col in df.columns if '/qualifierId' in col]

# get pass locations
df['passEndX'] = 0.0
df['passEndY'] = 0.0
for i, row in df.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 140:
                df.at[i, 'passEndX'] = float(q['value'])
            elif q.get('qualifierId') == 141:
                df.at[i, 'passEndY'] = float(q['value'])

# get pass length and angle
df['pass_angle'] = 0.0
df['pass_length'] = 0.0
for i, row in df.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 212:
                df.at[i, 'pass_length'] = float(q['value'])
            elif q.get('qualifierId') == 213:
                df.at[i, 'pass_angle'] = float(q['value'])

# get cutbacks and crosses
df['cutback'] = 0
#df['cross'] = 0
for i, row in df.iterrows():
    qualifier_str = row['qualifier']
    if pd.notna(qualifier_str):
        qualifiers = json.loads(qualifier_str.replace("'", '"'))  # Fix quotes for JSON
        for q in qualifiers:
            if q.get('qualifierId') == 195:
                df.at[i, 'cutback'] = float(q['value'])
            # elif q.get('qualifierId') == 2:
            #     df.at[i, 'cross'] = float(q['value'])

# get cross
df['cross'] = 0
for i in range(len(df)):
    df1 = df.iloc[i:i+1,:]
    j = 0
    for j in range(len(type_cols)):
        col = df1[type_cols[j]].values[0]
        if col == 2:
            df['cross'][i] = 1
        else:
            j +=1

# get through balls
df['through_ball'] = 0
for i in range(len(df)):
    df1 = df.iloc[i:i+1,:]
    j = 0
    for j in range(len(type_cols)):
        col = df1[type_cols[j]].values[0]
        if col == 4:
            df['through_ball'][i] = 1
        else:
            j +=1

# switch of play
df['switch_ball'] = 0
for i in range(len(df)):
    df1 = df.iloc[i:i+1,:]
    j = 0
    for j in range(len(type_cols)):
        col = df1[type_cols[j]].values[0]
        if col == 196:
            df['switch_ball'][i] = 1
        else:
            j +=1

In [65]:
df = df[df['typeId'] == 1]
df

,id,eventId,typeId,periodId,timeMin,timeSec,contestantId,outcome,x,y,...,Squad,Position,passEndX,passEndY,pass_angle,pass_length,cutback,cross,through_ball,switch_ball
4,2803502135,4,1,1,0,0,b4fbmos00gxtk8qnaqsm392p5,1,50.0,50.0,...,North Carolina Courage,Home,29.8,48.1,3.20,21.2,0,0,0,0
6,2803502141,5,1,1,0,6,b4fbmos00gxtk8qnaqsm392p5,0,29.8,47.0,...,North Carolina Courage,Home,72.7,57.9,0.16,45.7,0,0,0,0
7,2803502151,3,1,1,0,8,eka22bj7co8bw5qxa6qlbinf8,0,25.4,33.7,...,Bay,Away,36.2,29.7,6.05,11.7,0,0,0,0
8,2803548065,406,1,1,0,10,b4fbmos00gxtk8qnaqsm392p5,1,64.0,77.2,...,North Carolina Courage,Home,65.1,66.9,4.88,7.1,0,0,0,0
9,2803502221,6,1,1,0,12,b4fbmos00gxtk8qnaqsm392p5,1,61.2,62.6,...,North Carolina Courage,Home,41.6,68.1,2.96,20.9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2803590429,799,1,2,97,26,eka22bj7co8bw5qxa6qlbinf8,0,5.4,57.9,...,Bay,Away,55.8,84.5,0.33,55.9,0,0,0,0
1570,2803590453,759,1,2,97,32,b4fbmos00gxtk8qnaqsm392p5,1,44.4,26.0,...,North Carolina Courage,Home,64.1,8.2,5.75,24.0,0,0,0,0
1571,2803590457,760,1,2,97,34,b4fbmos00gxtk8qnaqsm392p5,0,64.1,8.2,...,North Carolina Courage,Home,83.4,18.8,0.34,21.5,0,0,0,0
1575,2803590563,762,1,2,97,41,b4fbmos00gxtk8qnaqsm392p5,1,81.3,0.0,...,North Carolina Courage,Home,75.1,10.4,2.28,10.0,0,0,0,0


In [39]:
# df_passes = df[df.typeId==1]
# # Define functions to check the presence of each qualifier
# def has_qualifier_long_balls(qualifiers):
#     return any(item['qualifierId'] == 1 for item in qualifiers)

# def has_qualifier_crosses(qualifiers):
#     return any(item['qualifierId'] == 2 for item in qualifiers)

# def has_qualifier_head_passes(qualifiers):
#     return any(item['qualifierId'] == 3 for item in qualifiers)

# def has_qualifier_through_balls(qualifiers):
#     return any(item['qualifierId'] == 4 for item in qualifiers)

# def has_qualifier_free_kicks(qualifiers):
#     return any(item['qualifierId'] == 5 for item in qualifiers)

# def has_qualifier_corners(qualifiers):
#     return any(item['qualifierId'] == 6 for item in qualifiers)

# def has_qualifier_attacking_passes(qualifiers):
#     return any(item['qualifierId'] == 106 for item in qualifiers)

# def has_qualifier_throw_ins(qualifiers):
#     return any(item['qualifierId'] == 107 for item in qualifiers)

# def has_qualifier_chipped_passes(qualifiers):
#     return any(item['qualifierId'] == 155 for item in qualifiers)

# def has_qualifier_lay_offs(qualifiers):
#     return any(item['qualifierId'] == 156 for item in qualifiers)

# def has_qualifier_launched_passes(qualifiers):
#     return any(item['qualifierId'] == 157 for item in qualifiers)

# def has_qualifier_flick_ons(qualifiers):
#     return any(item['qualifierId'] == 168 for item in qualifiers)

# def has_qualifier_pull_backs(qualifiers):
#     return any(item['qualifierId'] == 195 for item in qualifiers)

# def has_qualifier_switches_of_play(qualifiers):
#     return any(item['qualifierId'] == 196 for item in qualifiers)

# def has_qualifier_shot_assists(qualifiers):
#     return any(item['qualifierId'] == 210 for item in qualifiers)

# def has_qualifier_second_assists(qualifiers):
#     return any(item['qualifierId'] == 218 for item in qualifiers)

# # Apply each function to create new boolean columns
# df_passes['has_qualifier_long_balls'] = df_passes['qualifier'].apply(has_qualifier_long_balls)
# df_passes['has_qualifier_crosses'] = df_passes['qualifier'].apply(has_qualifier_crosses)
# df_passes['has_qualifier_head_passes'] = df_passes['qualifier'].apply(has_qualifier_head_passes)
# df_passes['has_qualifier_through_balls'] = df_passes['qualifier'].apply(has_qualifier_through_balls)
# df_passes['has_qualifier_free_kicks'] = df_passes['qualifier'].apply(has_qualifier_free_kicks)
# df_passes['has_qualifier_corners'] = df_passes['qualifier'].apply(has_qualifier_corners)
# df_passes['has_qualifier_attacking_passes'] = df_passes['qualifier'].apply(has_qualifier_attacking_passes)
# df_passes['has_qualifier_throw_ins'] = df_passes['qualifier'].apply(has_qualifier_throw_ins)
# df_passes['has_qualifier_chipped_passes'] = df_passes['qualifier'].apply(has_qualifier_chipped_passes)
# df_passes['has_qualifier_lay_offs'] = df_passes['qualifier'].apply(has_qualifier_lay_offs)
# df_passes['has_qualifier_launched_passes'] = df_passes['qualifier'].apply(has_qualifier_launched_passes)
# df_passes['has_qualifier_flick_ons'] = df_passes['qualifier'].apply(has_qualifier_flick_ons)
# df_passes['has_qualifier_pull_backs'] = df_passes['qualifier'].apply(has_qualifier_pull_backs)
# df_passes['has_qualifier_switches_of_play'] = df_passes['qualifier'].apply(has_qualifier_switches_of_play)
# df_passes['has_qualifier_shot_assists'] = df_passes['qualifier'].apply(has_qualifier_shot_assists)
# df_passes['has_qualifier_second_assists'] = df_passes['qualifier'].apply(has_qualifier_second_assists)

# # Function to extract endX and endY values
# def extract_end_coordinates(qualifiers):
#     endX = next((item['value'] for item in qualifiers if item['qualifierId'] == 140), None)
#     endY = next((item['value'] for item in qualifiers if item['qualifierId'] == 141), None)
#     return endX, endY

# # Apply the function to extract endX and endY values
# df_passes[['passEndX', 'passEndY']] = df_passes['qualifier'].apply(lambda x: pd.Series(extract_end_coordinates(x)))

# # Define functions to extract the values of "angle" and "length" qualifiers
# def extract_angle(qualifiers):
#     angle = next((item['value'] for item in qualifiers if item['qualifierId'] == 213), None)
#     return angle

# def extract_length(qualifiers):
#     length = next((item['value'] for item in qualifiers if item['qualifierId'] == 212), None)
#     return length

# # Apply each function to create new columns for "angle" and "length"
# df_passes['angle'] = df_passes['qualifier'].apply(extract_angle)
# df_passes['length'] = df_passes['qualifier'].apply(extract_length)

# #convert to relevant columns to numeric data
# df_passes['passEndX'] = pd.to_numeric(df_passes['passEndX'])
# df_passes['passEndY'] = pd.to_numeric(df_passes['passEndY'])
# df_passes['angle'] = pd.to_numeric(df_passes['angle'])
# df_passes['length'] = pd.to_numeric(df_passes['length'])

# # Drop the original 'qualifier' column
# df_passes.drop(columns=['qualifier'], inplace=True)